<a href="https://colab.research.google.com/github/njgeorge000158/Charity-Funding-Analysis-with-Neural-Networks-using-TensorFlow/blob/main/alphabet_soup_charity_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#*******************************************************************************************
 #
 #  File Name:  alphabet_soup_charity_optimization.ipynb
 #
 #  File Description:
 #      This interactive Python notebook, alphabet_soup_charity_optimization.ipynb,
 #      reads a csv file, charity_data.csv, in the folder, resources, and uses Python
 #      and deep learning methods to process the features in the provided dataset and
 #      create a binary classifier that can predict whether applicants for charity
 #      donations will be successful if funded by the nonprofit foundation, Alphabet Soup.
 #
 #      I reinstalled the scikit-learn module using the following commands to allow the
 #      RandomOverSampler function to work:
 #
 #      pip3 uninstall scikit-learn
 #      pip3 install scikit-learn==1.2.2
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  12/01/2023      Initial Development                     Nicholas J. George
 #
 #******************************************************************************************/

from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.insert(0,'/content/drive/My Drive/alphabet_soup_charity_analysis')

!pip install -q dataframe_image
!pip install hvplot
!pip install silence-tensorflow
!pip install keras-tuner

import log_subroutines
import pandas_process_functions

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import tensorflow as tf

from keras import callbacks

Mounted at /content/drive/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.1/398.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.2.1-py3-none-any.whl size=4465 sha256=7c1e0ec2790700edf2e343922710b3e53e31682753512341a410e7da93f10748
  Stored in directory: /root/.cache/pip/wheels/7d/2c/24/e130d6102c0df56631b9db7479d9a6a53c5d97fb06b5f61b98
  Created wheel for support_developer: filename=support_developer-1.0.5-py3-none-any.whl size=5630 sha256=4b95c2c37990b419a63f8d4cee843835c28002a5fcb094e100fdf264d8513866
  Stored in directory: /root/.cache/pip/wheels/b6/72/c8/3054a5897ba0713dfa7a941364d68cbd42b0755c8e2ec1c18c
Successfully built silence-tensorflow support_developer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
CONSTANT_LOCAL_FILE_NAME = 'alphabet_soup_charity.ipynb'

CONSTANT_CHARITY_DATA_CSV_FILE_PATH = '/content/drive/My Drive/alphabet_soup_charity_analysis/resources/charity_data.csv'

CONSTANT_FIRST_OUTPUT_FILE_PATH \
  = '/content/drive/My Drive/alphabet_soup_charity_analysis/resources/alphabet_soup_charity_optimization.keras'

# <br> **Section 1: Extraction**

## **1.1: Import and Read Charity Data**

In [3]:
data_type_dictionary \
    = {'EIN': str,
       'NAME':	str,
       'APPLICATION_TYPE': str,
       'AFFILIATION': str,
       'CLASSIFICATION': str,
       'USE_CASE': str,
       'ORGANIZATION': str,
       'STATUS': int,
       'INCOME_AMT': str,
       'SPECIAL_CONSIDERATIONS': str,
       'ASK_AMT': int,
       'IS_SUCCESSFUL': int}

charity_dataframe \
    = pd.read_csv(CONSTANT_CHARITY_DATA_CSV_FILE_PATH, dtype = data_type_dictionary)

log_subroutines.log_write_object(charity_dataframe)

## **1.2: Display Charity Data**

In [4]:
pandas_process_functions.return_formatted_table \
    (charity_dataframe, 'Table 1.2.1: Charity Data Table')

EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,"5,000",1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,"108,590",1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,"6,692",1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,"142,590",1
10556855,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,1,0,N,"5,000",1
10558440,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,"31,452",1
10566033,ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,"7,508,025",1
10570430,ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,"94,389",1
10571689,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0


# <br> **Section 2: Preprocessing**

## **2.1: Drop the Non-Beneficial ID Columns**

In [5]:
charity_dataframe.drop(['EIN', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis = 1, inplace = True)

log_subroutines.log_write_object(charity_dataframe)

In [6]:
pandas_process_functions.return_formatted_table \
    (charity_dataframe, 'Table 1.2.2: Modified Charity Data Table')

NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,"5,000",1
AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,"108,590",1
ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,"5,000",0
SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,"6,692",1
GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,"142,590",1
MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,0,"5,000",1
FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,100000-499999,"31,452",1
ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,10M-50M,"7,508,025",1
ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1-9999,"94,389",1
INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,"5,000",0


## **2.2: Determine the Number of Unique Values in Each Column**

In [7]:
unique_value_count_each_column_series = charity_dataframe.nunique()

log_subroutines.log_write_object(unique_value_count_each_column_series)

In [8]:
log_subroutines.print_and_log_text(str(unique_value_count_each_column_series))

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64


## **2.3: NAME Binning**

### **Look at NAME Value Counts for Binning**

In [9]:
name_value_counts_for_binning_series \
    = charity_dataframe['NAME'].value_counts().sort_values(ascending = False)

name_value_counts_for_binning_series.name = 'NAME'

log_subroutines.log_write_object(name_value_counts_for_binning_series)

### **Choose a Cutoff Value and Create a List of Names to be Replaced**

In [10]:
name_cutoff_integer = 2

names_to_replace \
    = list(name_value_counts_for_binning_series \
            [name_value_counts_for_binning_series < name_cutoff_integer].index)

log_subroutines.log_write_object(names_to_replace)

### **Replace Application Types in DataFrame**

In [11]:
for name in names_to_replace:

    charity_dataframe['NAME'] = charity_dataframe['NAME'].replace(name, 'Other')

### **Check to Make Sure Binning was Successful**

In [12]:
name_value_counts_for_binning_series \
    = charity_dataframe['NAME'].value_counts().sort_values(ascending = False)

name_value_counts_for_binning_series.name = 'NAME'

log_subroutines.log_write_object(name_value_counts_for_binning_series)

## **2.4: APPLICATION_TYPE Binning**

### **Look at APPLICATION_TYPE Value Counts for Binning**

In [13]:
application_type_value_counts_for_binning_series \
    = charity_dataframe['APPLICATION_TYPE'].value_counts().sort_values(ascending = False)

application_type_value_counts_for_binning_series.name = 'APPLICATION_TYPE'

log_subroutines.log_write_object(unique_value_count_each_column_series)

In [14]:
log_subroutines.print_and_log_text(str(application_type_value_counts_for_binning_series))

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


### **Choose a Cutoff Value and Create a List of Application Types to be Replaced**

In [15]:
application_type_cutoff_integer = 157

application_types_to_replace \
    = list(application_type_value_counts_for_binning_series \
            [application_type_value_counts_for_binning_series < application_type_cutoff_integer].index)

log_subroutines.log_write_object(application_types_to_replace)

In [16]:
log_subroutines.print_and_log_text(str(application_types_to_replace))

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


### **Replace Application Types in DataFrame**

In [17]:
for application_type in application_types_to_replace:

    charity_dataframe['APPLICATION_TYPE'] \
        = charity_dataframe['APPLICATION_TYPE'].replace(application_type, 'Other')

### **Check to Make Sure Binning was Successful**

In [18]:
application_type_value_counts_for_binning_series \
    = charity_dataframe['APPLICATION_TYPE'].value_counts().sort_values(ascending = False)

application_type_value_counts_for_binning_series.name = 'APPLICATION_TYPE'

log_subroutines.log_write_object(application_type_value_counts_for_binning_series)

In [19]:
log_subroutines.print_and_log_text(str(application_type_value_counts_for_binning_series))

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


## **2.5: AFFILIATION Binning**

### **Look at AFFILIATION Value Counts for Binning**

In [20]:
affiliation_value_counts_for_binning_series \
    = charity_dataframe['AFFILIATION'].value_counts().sort_values(ascending = False)

affiliation_value_counts_for_binning_series.name = 'AFFILIATION'

log_subroutines.log_write_object(affiliation_value_counts_for_binning_series)

In [21]:
log_subroutines.print_and_log_text(str(affiliation_value_counts_for_binning_series))

AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64


### **Choose a Cutoff Value and Create a List of Affiliations to be Replaced**

In [22]:
affiliation_cutoff_integer = 65

affiliations_to_replace \
    = list(affiliation_value_counts_for_binning_series \
            [affiliation_value_counts_for_binning_series < affiliation_cutoff_integer].index)

log_subroutines.log_write_object(affiliations_to_replace)

In [23]:
log_subroutines.print_and_log_text(str(affiliations_to_replace))

['Family/Parent', 'National', 'Regional', 'Other']


### **Replace Affiliations in DataFrame**

In [24]:
for affiliation in affiliations_to_replace:

    charity_dataframe['AFFILIATION'] = charity_dataframe['AFFILIATION'].replace(affiliation, 'Other')

### **Check to Make Sure Binning was Successful**

In [25]:
affiliation_value_counts_for_binning_series \
    = charity_dataframe['AFFILIATION'].value_counts().sort_values(ascending = False)

affiliation_value_counts_for_binning_series.name = 'AFFILIATION'

log_subroutines.log_write_object(affiliation_value_counts_for_binning_series)

In [26]:
log_subroutines.print_and_log_text(str(affiliation_value_counts_for_binning_series))

AFFILIATION
Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64


## **2.6: CLASSIFICATION Binning**

### **Look at CLASSIFICATION Value Counts for Binning**

In [27]:
classification_value_counts_for_binning_series \
    = charity_dataframe['CLASSIFICATION'].value_counts().sort_values(ascending = False)

classification_value_counts_for_binning_series.name = 'CLASSIFICATION'

log_subroutines.log_write_object(classification_value_counts_for_binning_series)

In [28]:
log_subroutines.print_and_log_text(str(classification_value_counts_for_binning_series))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C2600        1
C4200        1
C2190        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


### **Look at CLASSIFICATION Value Counts > 1**

In [29]:
classification_value_counts_for_binning_series[classification_value_counts_for_binning_series > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

### **Choose a Cutoff Value and Create a List of Classifications to be Replaced**

In [30]:
classification_cutoff_integer = 96

classifications_to_replace \
    = list(classification_value_counts_for_binning_series \
            [classification_value_counts_for_binning_series < classification_cutoff_integer].index)

log_subroutines.log_write_object(classifications_to_replace)

### **Replace Classifications in DataFrame**

In [31]:
for classification in classifications_to_replace:

    charity_dataframe['CLASSIFICATION'] \
        = charity_dataframe['CLASSIFICATION'].replace(classification, 'Other')

### **Check to Make Sure Binning was Successful**

In [32]:
classification_value_counts_for_binning_series \
    = charity_dataframe['CLASSIFICATION'].value_counts().sort_values(ascending = False)

classification_value_counts_for_binning_series.name = 'CLASSIFICATION'

log_subroutines.log_write_object(classification_value_counts_for_binning_series)

In [33]:
log_subroutines.print_and_log_text(str(classification_value_counts_for_binning_series))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64


## **2.7: USE_CASE Binning**

### **Look at USE_CASE Value Counts for Binning**

In [34]:
use_case_value_counts_for_binning_series \
    = charity_dataframe['USE_CASE'].value_counts().sort_values(ascending = False)

use_case_value_counts_for_binning_series.name = 'USE_CASE'

log_subroutines.log_write_object(use_case_value_counts_for_binning_series)

In [35]:
log_subroutines.print_and_log_text(str(use_case_value_counts_for_binning_series))

USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64


### **Choose a Cutoff Value and Create a List of Use Cases to be Replaced**

In [36]:
use_case_cutoff_integer = 147

use_cases_to_replace \
    = list(use_case_value_counts_for_binning_series \
            [use_case_value_counts_for_binning_series < use_case_cutoff_integer].index)

log_subroutines.log_write_object(use_cases_to_replace)

### **Replace Use Cases in DataFrame**

In [37]:
for use_case in use_cases_to_replace:

    charity_dataframe['USE_CASE'] = charity_dataframe['USE_CASE'].replace(use_case, 'Other')

### **Check to Make Sure Binning was Successful**

In [38]:
use_case_value_counts_for_binning_series \
    = charity_dataframe['USE_CASE'].value_counts().sort_values(ascending = False)

use_case_value_counts_for_binning_series.name = 'USE_CASE'

log_subroutines.log_write_object(use_case_value_counts_for_binning_series)

In [39]:
log_subroutines.print_and_log_text(str(use_case_value_counts_for_binning_series))

USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Other              149
Name: USE_CASE, dtype: int64


## **2.8: ASK_AMT Binning**

### **Look at ASK_AMT Value Counts for Binning**

In [40]:
ask_amt_value_counts_for_binning_series \
    = charity_dataframe['ASK_AMT'].value_counts().sort_values(ascending = False)

ask_amt_value_counts_for_binning_series.name = 'ASK_AMT'

log_subroutines.log_write_object(ask_amt_value_counts_for_binning_series)

In [41]:
log_subroutines.print_and_log_text(str(ask_amt_value_counts_for_binning_series))

ASK_AMT
5000        25398
15583           3
63981           3
6725            3
10478           3
            ...  
11336           1
1270766         1
121619          1
46915           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


### **Choose a Cutoff Value and Create a List of Ask Amounts to be Replaced**

In [42]:
ask_amt_cutoff_integer = 4

ask_amounts_to_replace \
    = list(ask_amt_value_counts_for_binning_series \
            [ask_amt_value_counts_for_binning_series < ask_amt_cutoff_integer].index)

log_subroutines.log_write_object(ask_amounts_to_replace)

### **Replace Ask Amounts in DataFrame**

In [43]:
for ask_amt in ask_amounts_to_replace:

    charity_dataframe['ASK_AMT'] \
        = charity_dataframe['ASK_AMT'].replace(ask_amt, 'Other')

### **Check to Make Sure Binning was Successful**

In [44]:
ask_amt_value_counts_for_binning_series \
    = charity_dataframe['ASK_AMT'].value_counts().sort_values(ascending = False)

ask_amt_value_counts_for_binning_series.name = 'ASK_AMT'

log_subroutines.log_write_object(ask_amt_value_counts_for_binning_series)

In [45]:
log_subroutines.print_and_log_text(str(ask_amt_value_counts_for_binning_series))

ASK_AMT
5000     25398
Other     8901
Name: ASK_AMT, dtype: int64


## **2.9: Final Preprocessing**

### **Convert Categorical Data to Numeric with `pd.get_dummies`**

In [46]:
dummies_dataframe = pd.get_dummies(charity_dataframe)

log_subroutines.log_write_object(dummies_dataframe)

In [47]:
pandas_process_functions.return_formatted_table \
    (dummies_dataframe, 'Table 2.5.1: Dummies DataFrame Table')

IS_SUCCESSFUL,NAME_4-H CLUBS & AFFILIATED 4-H ORGANIZATIONS,NAME_AACE INTERNATIONAL,NAME_ACADEMY OF GENERAL DENTISTRY,NAME_ACADEMY OF MANAGED CARE PHARMACY,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTIVE 20-30 UNITED STATES AND CANADA,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AGENTS ASSOCIATION,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALASKA PTA,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,NAME_ALLIANCE FOR RETIRED AMERICANS,NAME_ALLIANCE OF DIVINE LOVE INC,NAME_ALLIANCE OF THE FRIENDS OF THE DEKALB COUNTY LIBRARY,NAME_ALPHA DELTA KAPPA SORORITY INC,NAME_ALPHA GAMMA RHO FRATERNITY,NAME_ALPHA KAPPA PSI FRATERNITY,NAME_ALPHA PHI OMEGA,NAME_ALPHA PHI SIGMA,NAME_ALPHA PHI SIGMA INC,NAME_ALPHA SIGMA PHI FRATERNITY INC,NAME_ALPHA TAU OMEGA FRATERNITY,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,NAME_AMALGAMATED TRANSIT UNION,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMATEUR ATHLETIC UNION OV THE UNITED STATES INC,NAME_AMATEUR ATHLETIC UNIONS OF THE US,NAME_AMERCIAN LEGION,NAME_AMERICAN AGAPE FOUNDATION INC,NAME_AMERICAN ART THERAPY ASSOCIATION,NAME_AMERICAN ART THERAPY ASSOCIATION INC,NAME_AMERICAN ASSOCIATION OF FAMILY AND CONSUMER SCIENCES,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN INC,NAME_AMERICAN ASSOCIATION OF ZOO KEEPERS,NAME_AMERICAN BEGONIA SOCIETY,NAME_AMERICAN BUSINESS WOMENS ASSOCIATION,NAME_AMERICAN CANCER SOCIETY INC,NAME_AMERICAN CHEMICAL SOCIETY,NAME_AMERICAN CHRISTIAN FICTION WRITERS INC,NAME_AMERICAN COACH ASSOCIATION,NAME_AMERICAN COLLEGE OF HEALTHCARE EXECUTIVES,NAME_AMERICAN COLLEGE OF NURSE-MIDWIVES,NAME_AMERICAN COLLEGE OF PHYSICIANS,NAME_AMERICAN COLLEGE OF PHYSICIANS SERVICES,NAME_AMERICAN CRIMINAL JUSTICE ASSOCIATION LAMBDA ALPHA EPSILON,NAME_AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES,NAME_AMERICAN FEDERATION OF LABOR & CONGRESS OF INDUSTRIAL ORGS,NAME_AMERICAN FEDERATION OF LABOR AND CONGRESS OF INDUSTRIAL ORGS,NAME_AMERICAN FEDERATION OF STATE COUNTY & MUNICIPAL EMPLOYEES,NAME_AMERICAN FEDERATION OF TEACHERS,NAME_AMERICAN FISHERIES SOCIETY,NAME_AMERICAN GROUP PSYCHOTHERAPY ASSOCIATION INC,NAME_AMERICAN HARP SOCIETY,NAME_AMERICAN HIBISCUS,NAME_AMERICAN HUMANIST ASSOCIATION,NAME_AMERICAN INDEPENDENT BUSINESS ALLIANCE,NAME_AMERICAN INSTITUTE OF GRAPHIC ARTS,NAME_AMERICAN IRIS SOCIETY,NAME_AMERICAN LEGION,NAME_AMERICAN LEGION AUXILIARY,NAME_AMERICAN LEGION POST,NAME_AMERICAN MARKETING ASSOCIATION INC,NAME_AMERICAN NEPHROLOGY NURSES ASSOCIATION,NAME_AMERICAN ORFF-SCHULWERK ASSOCIATION,NAME_AMERICAN POSTAL WORKERS UNION,NAME_AMERICAN RESEARCH CENTER IN EGYPT,NAME_AMERICAN RHODODENDRON SOCIETY,NAME_AMERICAN ROSE SOCIETY,NAME_AMERICAN SOCIETY OF ADDICTION MEDICINE INC,NAME_AMERICAN SOCIETY OF HEATING REFRIGERATING & A-C ENGINEERS,NAME_AMERICAN STUDENT DENTAL ASSOCIATION,NAME_AMERICAN TRAIN DISPATCHERS ASSOCIATION,NAME_AMERICAN VOLKSSPORT ASSOCIATION INC,NAME_AMERICAN WATER WORKS ASSOCIATION,NAME_AMERICAN YOUTH FOOTBALL INC,NAME_AMERICAN-TURKISH ASSOCIATION OF SOUTHERN CALIFORNIA,NAME_AMERICANS UNITED FOR SEPARATION OF CHURCH & STATE,NAME_AMERICANS UNITED FOR SEPARATION OF CHURCH AND STATE,NAME_AMIGOS DE LAS AMERICAS,NAME_AMVETS,NAME_ANCHORAGE COMMUNITY MENTAL HEALTH SERVICES,NAME_ANCIENT AND FREE ACCEPTED MASONS OF MISSOURI,NAME_ANCIENT EGYPTIAN ARABIC ORDER NOBLES MYSTIC SHRINE OF N & S AMER,NAME_ANCIENT FREE & ACCEPTED MASON OF KANSAS,NAME_ANCIENT FREE & ACCEPTED MASONS OF COLORADO,NAME_ANCIENT FREE & ACCEPTED MASONS OF KANSAS,NAME_ANCIENT FREE & ACCEPTED MASONS OF OREGON,NAME_ANCIENT FREE AND ACCEPTED MASONS OF OKLAHOMA,NAME_ANCIENT ORDER OF HIBERNIANS IN AMERICA,NAME_APPA ASSOCIATION OF HIGHER EDUCATION FACILITIES OFFICERS,NAME_APPA-ASSOCIATION OF HIGHER EDUCATION FACILITIES OFFICERS,NAME_ARC OF ARIZONA INC,NAME_ARIZONA FEDERATION OF GARDEN CLUBS,NAME_

### **Split Preprocessed Data into Features and Target Arrays**

In [48]:
y_nparray = dummies_dataframe['IS_SUCCESSFUL'].values

log_subroutines.log_write_object(y_nparray)

In [49]:
x_nparray = dummies_dataframe.drop(['IS_SUCCESSFUL'], axis = 1).values

log_subroutines.log_write_object(x_nparray)

### **Split the Preprocessed Data into a Training and Testing Dataset**

In [50]:
x_train_nparray, x_test_nparray, y_train_nparray, y_test_nparray \
    = train_test_split(x_nparray, y_nparray, random_state = 21)

In [51]:
log_subroutines.log_write_object(str(x_train_nparray) + '\n\n')

log_subroutines.log_write_object(str(x_test_nparray) + '\n\n')

log_subroutines.log_write_object(str(y_train_nparray) + '\n\n')

log_subroutines.log_write_object(str(y_test_nparray) + '\n\n')

### **Create a StandardScaler Instance**

In [52]:
current_standard_scalar = StandardScaler()

### **Fit the StandardScaler**

In [53]:
x_standard_scalar = current_standard_scalar.fit(x_train_nparray)

### **Scale the Data**

In [54]:
x_train_scaled_nparray = x_standard_scalar.transform(x_train_nparray)

log_subroutines.log_write_object(x_train_scaled_nparray)

In [55]:
x_test_scaled_nparray = x_standard_scalar.transform(x_test_nparray)

log_subroutines.log_write_object(x_test_scaled_nparray)

# <br> **Section 3: Compile, Train, Evaluate, and Export the Model**

## **3.1: Compile Model**

### **Model Definition**

In [56]:
neural_net_sequential = tf.keras.models.Sequential()

### **Layers**

In [57]:
input_dimension_integer = len(x_train_scaled_nparray[0])

# This line of code is the first layer.
neural_net_sequential.add \
    (tf.keras.layers.Dense(units = 53, activation = 'relu', input_dim = input_dimension_integer))

# This line of code is the hidden layer.
neural_net_sequential.add(tf.keras.layers.Dense(units = 20, activation = 'relu'))

# This line of code is the output layer.
neural_net_sequential.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### **Check Model**

In [58]:
neural_net_sequential.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 53)                44361     
                                                                 
 dense_1 (Dense)             (None, 20)                1080      
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 45462 (177.59 KB)
Trainable params: 45462 (177.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### **Compile**

In [59]:
neural_net_sequential.compile \
    (loss = 'log_cosh',
     optimizer = tf.keras.optimizers.experimental.Adamax(learning_rate = 0.00088933),
     metrics = ['accuracy'])

## **3.2: Train Model**

In [60]:
earlystopping_callback \
    = callbacks.EarlyStopping \
        (monitor = 'val_accuracy', mode = 'max', patience = 42, restore_best_weights = True)

neural_net_sequential \
    .fit \
        (x_train_scaled_nparray,
         y_train_nparray,
         epochs = 1000,
         validation_data = (x_test_scaled_nparray, y_test_nparray),
         callbacks = [earlystopping_callback])

Epoch 1/1000
804/804 [==============================] - 4s 4ms/step - loss: 0.0852 - accuracy: 0.7472 - val_loss: 0.0725 - val_accuracy: 0.7867
Epoch 2/1000
804/804 [==============================] - 2s 3ms/step - loss: 0.0706 - accuracy: 0.7886 - val_loss: 0.0685 - val_accuracy: 0.7949
Epoch 3/1000
804/804 [==============================] - 2s 3ms/step - loss: 0.0672 - accuracy: 0.7959 - val_loss: 0.0670 - val_accuracy: 0.7949
Epoch 4/1000
804/804 [==============================] - 2s 3ms/step - loss: 0.0655 - accuracy: 0.8001 - val_loss: 0.0659 - val_accuracy: 0.8000
Epoch 5/1000
804/804 [==============================] - 3s 3ms/step - loss: 0.0644 - accuracy: 0.8036 - val_loss: 0.0652 - val_accuracy: 0.8027
Epoch 6/1000
804/804 [==============================] - 3s 4ms/step - loss: 0.0636 - accuracy: 0.8055 - val_loss: 0.0647 - val_accuracy: 0.8042
Epoch 7/1000
804/804 [==============================] - 2s 3ms/step - loss: 0.0631 - accuracy: 0.8066 - val_loss: 0.0646 - val_accuracy:

## **3.3: Evaluate Model**

In [61]:
model_loss_float, model_accuracy_float \
    = neural_net_sequential.evaluate(x_test_scaled_nparray, y_test_nparray, verbose = 2)

log_subroutines \
    .print_and_log_text \
        (f'\nModel Loss: {round(model_loss_float * 100, 2)}%, '
         + f'Model Accuracy: {round(model_accuracy_float * 100, 2)}%')

268/268 - 0s - loss: 0.0636 - accuracy: 0.8094 - 362ms/epoch - 1ms/step

Model Loss: 6.36%, Model Accuracy: 80.94%


## **3.4: Export Model**

In [62]:
neural_net_sequential.save(CONSTANT_FIRST_OUTPUT_FILE_PATH)